In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam
from sklearn.preprocessing import MinMaxScaler

# Define TCN components
class CausalConv1d(nn.Conv1d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True):
        super().__init__(in_channels, out_channels, kernel_size, stride=stride, padding=0, dilation=dilation, groups=groups, bias=bias)
        self.__padding = (kernel_size - 1) * dilation

    def forward(self, x):
        return super().forward(nn.functional.pad(x, (self.__padding, 0)))

class TCNBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation):
        super().__init__()
        self.conv1 = CausalConv1d(in_ch, out_ch, kernel_size, dilation=dilation)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.conv2 = CausalConv1d(out_ch, out_ch, kernel_size, dilation=dilation)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.downsample = nn.Conv1d(in_ch, out_ch, 1) if in_ch != out_ch else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.dropout1(self.relu1(self.conv1(x)))
        out = self.dropout2(self.relu2(self.conv2(out)))
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TCN(nn.Module):
    def __init__(self, input_size=1, num_channels=[32]*7, kernel_size=3, horizon=72, dropout=0.2):
        super().__init__()
        self.layers = nn.ModuleList()
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation = 2 ** i
            in_ch = input_size if i == 0 else num_channels[i-1]
            out_ch = num_channels[i]
            self.layers.append(TCNBlock(in_ch, out_ch, kernel_size, dilation))
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(num_channels[-1], horizon)  # Output horizon values directly

    def forward(self, x):
        # x: (batch, features=1, seq_len=lookback)
        for layer in self.layers:
            x = layer(x)
        # Take the last time step: (batch, channels)
        x = x[:, :, -1]
        x = self.dropout(x)
        x = self.linear(x)  # (batch, horizon)
        return x

# Parameters
lookback = 168  # 7 days
horizon = 72    # 3 days
epochs = 50
batch_size = 32
learning_rate = 0.001

# Read input
df = pd.read_csv('merged_output2.csv')
df['DATE_MILADI'] = pd.to_datetime(df['DATE_MILADI'])
df = df.sort_values(['UNIT_NO', 'DATE_MILADI', 'HOUR']).reset_index(drop=True)

# Output df: only 2024+
test_df = df[df['DATE_MILADI'].dt.year >= 2024].copy()
test_df['DECLARED'] = np.nan

# Group by UNIT_NO
for unit_no, group in df.groupby('UNIT_NO'):
    group = group.reset_index(drop=True)
    
    # Train: 2021-2023
    train_mask = (group['DATE_MILADI'].dt.year >= 2021) & (group['DATE_MILADI'].dt.year <= 2023)
    train_part = group[train_mask]
    
    if len(train_part) < lookback + horizon:
        print(f"Skipping UNIT_NO {unit_no}: داده آموزشی کافی نیست")
        continue
    
    # Scaler on train
    scaler = MinMaxScaler()
    scaler.fit(train_part['POWER'].values.reshape(-1, 1))
    
    # Scale all
    scaled_power = scaler.transform(group['POWER'].values.reshape(-1, 1)).flatten()
    
    # Sequences
    X, y = [], []
    for i in range(len(scaled_power) - lookback - horizon + 1):
        X.append(scaled_power[i:i + lookback])
        y.append(scaled_power[i + lookback : i + lookback + horizon])
    X = np.array(X)
    y = np.array(y)
    
    if len(X) == 0:
        continue
    
    # Tensors
    X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # (samples, 1, lookback)
    y_tensor = torch.tensor(y, dtype=torch.float32)  # (samples, horizon)
    
    # Dataset
    dataset = TensorDataset(X_tensor, y_tensor)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Model (deeper for better receptive field)
    model = TCN(input_size=1, num_channels=[32]*7, kernel_size=3, horizon=horizon)
    optimizer = Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()
    
    # Train
    model.train()
    for epoch in range(epochs):
        for batch_X, batch_y in loader:
            optimizer.zero_grad()
            output = model(batch_X)  # (batch, horizon)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()
    
    # Predict for test (2024+)
    test_start_idx = group[train_mask].index[-1] + 1
    
    model.eval()
    with torch.no_grad():
        for current_idx in range(max(test_start_idx - lookback, 0), len(group)):
            if current_idx + horizon <= test_start_idx:
                continue  # Skip old predictions
            
            input_seq = scaled_power[current_idx - lookback : current_idx]
            if len(input_seq) < lookback:
                continue
            
            input_tensor = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # (1,1,lookback)
            
            pred_scaled = model(input_tensor).numpy()  # (1, horizon) -> (horizon,)
            pred_scaled = pred_scaled.flatten()
            
            # Assign to future rows
            end_pred = min(current_idx + horizon, len(group))
            num_assign = end_pred - current_idx
            if current_idx < test_start_idx and end_pred > test_start_idx:
                # Overlap: only assign from test_start_idx
                start_assign = test_start_idx - current_idx
                declared_values = scaler.inverse_transform(pred_scaled[start_assign:start_assign + (end_pred - test_start_idx)].reshape(-1, 1)).flatten()
                assign_indices = group.index[test_start_idx : end_pred]
            elif current_idx >= test_start_idx:
                declared_values = scaler.inverse_transform(pred_scaled[:num_assign].reshape(-1, 1)).flatten()
                assign_indices = group.index[current_idx : end_pred]
            else:
                continue
            
            if len(assign_indices) > 0:
                test_df.loc[assign_indices, 'DECLARED'] = declared_values

# Apply rule
test_df.loc[test_df['ebraz'] == 0, 'DECLARED'] = 0

# Fill any remaining NaN in DECLARED (if not covered)
test_df['DECLARED'] = test_df['DECLARED'].fillna(0)  # Or handle differently if needed

# Add features
test_df['year'] = test_df['DATE_MILADI'].dt.year
test_df['month'] = test_df['DATE_MILADI'].dt.month
test_df['dayofweek'] = test_df['DATE_MILADI'].dt.dayofweek

# Columns order
columns = [
    'HOUR', 'DATE_MILADI', 'DATE_SHAMSI', 'POWER', 'CODE', 'UNIT_NO', 'DAMA', 'ROTOOBAT',
    '12209_G13', '12210_G13', 'ebraz', 'importance_factor', 'year', 'month', 'dayofweek', 'DECLARED'
]
output_df = test_df[columns]

# Save
output_df.to_csv('output.csv', index=False)
output_df.to_excel('output.xlsx', index=False)

print("پیش‌بینی با TCN تکمیل شد. خروجی فقط برای سال 2024 و بعد ذخیره شد.")
print("فایل‌ها: output.csv و output.xlsx")

In [ ]:
# pip install torch

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/110.9 MB 316.6 kB/s eta 0:05:49
   ---------------------------------------- 0.5/110.9 MB 316.6 kB/s eta 0:05:49
   ---------------------------------------- 0.5/110.9 MB 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
